# Version 2.4 – Nonlinear RHS Support: NLSE and Burgers' Equation

In this notebook, we demonstrate two new features introduced in version 2.4 of `utility-sim-tools`:

- Modular RHS support for nonlinear PDEs
- Explicit time integration of NLSE (nonlinear Schrödinger) and viscous Burgers' equation in 2D

This marks a shift from linear-only systems (v2.3) toward a general PDE simulation engine.

## Design Changes in v2.4

- Introduced `BasePDESystem2D` and `ExplicitPDESystem2D` to support flexible PDE evolution.
- Replaced `LinearPDESystem2D` with modular RHS factories (e.g. `make_linear_rhs`, `make_nlse_rhs`).
- Added `gradient_2d.py` for computing ∇u using central differences.
- Added nonlinear RHS construction for:
  - NLSE: `∂u/∂t = i (Δu + |u|² u)`
  - Burgers': `∂u/∂t = -u·∇u + ν Δu`

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath("../src"))
from core.pde_systems import ExplicitPDESystem2D
from core.rhs_examples import make_nlse_rhs, make_burgers_rhs
from numerics.laplacian_2d import make_laplacian_2d
from numerics.gradient_2d import make_gradient_2d
from visualization.plotting_2d import animate_2d

## Domain Setup

We define a 2D periodic square grid, initialize a Gaussian bump, and prepare finite difference operators for ∇²u and ∇u.

In [2]:
N = 64
L = 10.0
dx = dy = L / N
x = np.linspace(-L/2, L/2, N, endpoint=False)
y = x.copy()
X, Y = np.meshgrid(x, y, indexing="ij")

# Initial condition: Gaussian bump
u0 = np.exp(-(X**2 + Y**2) / 2)

## 1. NLSE Simulation

We evolve the NLSE using:

\[
\frac{\partial u}{\partial t} = \alpha \nabla^2 u + \beta |u|^2 u
\]

with parameters `α=1.0`, `β=5.0`. This demonstrates nonlinearity using only linear matrix ops and NumPy broadcasting.

In [3]:
L_op = make_laplacian_2d(N, N, dx, dy)
rhs_nlse = make_nlse_rhs(L_op, alpha=1.0, beta=5.0)
nlse_system = ExplicitPDESystem2D(rhs_nlse)

u_nlse = nlse_system.evolve(u0, dt=0.01, steps=200)
animate_2d(u_nlse, x, y, interval=40)

/Users/linearityproductions/Desktop/utility-sim-tools/src/core/rhs_examples.py:7: RuntimeWarning: overflow encountered in square
  nonlinear_term = beta * np.abs(u)**2 * u
/Users/linearityproductions/Desktop/utility-sim-tools/src/core/rhs_examples.py:7: RuntimeWarning: overflow encountered in multiply
  nonlinear_term = beta * np.abs(u)**2 * u
/Users/linearityproductions/Desktop/utility-sim-tools/src/core/rhs_examples.py:8: RuntimeWarning: invalid value encountered in add
  return (linear_term + nonlinear_term).flatten()


## 2. Burgers' Equation Simulation

We now simulate viscous Burgers’ equation:

\[
\frac{\partial u}{\partial t} = - u \cdot \nabla u + \nu \nabla^2 u
\]

This tests the gradient module, Laplacian operator, and nonlinear coupling in the RHS. The system should exhibit dissipative smoothing.

In [4]:
grad_func = make_gradient_2d(N, N, dx, dy)
rhs_burgers = make_burgers_rhs(L_op, grad_func, nu=0.1)
burgers_system = ExplicitPDESystem2D(rhs_burgers)

u_burgers = burgers_system.evolve(u0, dt=0.01, steps=200)
animate_2d(u_burgers, x, y, interval=40)

## Summary

Version 2.4 established the foundation for nonlinear and custom PDEs.
- Future extensions (v2.5+) will introduce:
  - RK4 and Crank–Nicolson time integrators
  - Boundary condition modularity
  - Complex-valued fields and ITE support

We are now ready to simulate a broader class of scientific systems using clean Python components.